In [ ]:
class GNG:
    def __init__(self, X):
        self.X = X
        self.samples, features = self.X.shape
        minn = self.X.min()
        maxx = self.X.max()
        self.G = nx.Graph()
        self.first_two(minn, maxx, features)
        self.new_node = 2
                
        self.epochs = 100
        self.lambda = 30
        
        self.eta_f = 0.1
        self.eta_n = self.eta_f / 100
        self.max_age = 100
        self.biggest_error_reduction = 2
        self.global_error_reduction = 2
    
    def first_two(self, minn, maxx, features):
        for i in range(2):
            self.G.add_node(i, error = 0, weight = np.random.uniform(minn, maxx, (features)))
        self.G.add_edge(0, 1, age = 0)

    def training(self, epochs, lam):
        epoch = 0
        while epoch < epochs:
            epoch += 1
            sys.stdout.write('\r[%-50s] %d%%, epochs %d, nodes %d' % ('='*int(50 * epoch / epochs), 
                                                        100 * epoch / epochs, epoch, len(self.G.nodes)))
            sys.stdout.flush()
            order = np.random.permutation(self.samples)
            for i, ind in enumerate(order):
                self.update_net(ind)
                if i % lam == lam - 1:
                    self.create_neuron()
        sys.stdout.write('\n')
        
    def create_neuron(self):
        f = self.biggest_error(self.G.nodes)
        neighbors = [n for n in self.G[f]]
        s = self.biggest_error(neighbors)
        self.G.nodes[f]['error'] = self.G.nodes[f]['error'] / self.biggest_error_reduction
        self.G.nodes[s]['error'] = self.G.nodes[s]['error'] / self.biggest_error_reduction
        new_weight = (self.G.nodes[f]['weight'] + self.G.nodes[s]['weight']) / 2
        new_error = self.Gnodes[f]['error']
        self.G.add_node(self.new_node, error = new_error, weight = new_weight)
        self.G.add_edge(f, self.new_node, age = 0)
        self.G.add_edge(s, self.new_node, age = 0)
        self.G.remove_edge(f, s)
        self.new_node = self.new_node + 1
        
    def biggest_error(self, nodes):
        max_error = -1
        for er in nodes:
            error = self.G.nodes[er]['error'] / self.global_error_reduction
            self.G.nodes[er]['error'] = error
            if error > max_error:
                target = er
                max_error = error
        return target

    def update_net(self, ind):
        x = self.X[ind]
        min_0 = np.inf
        weights = []
        ind_to_node = []
        node_to_ind = {}
        for i, n in enumerate(self.G.nodes):
            ind_to_node.append(n)
            node_to_ind[n] = i
            weights.append(self.G.nodes[n]['weight'])
        # calculate distance and find 2 smallest
        diff = x - np.asarray(weights)
        dist = np.einsum('ij,ij->i', diff, diff)
        fi, si = np.argpartition(dist, 1)[:2]
        f = ind_to_node[fi]
        s = ind_to_node[si]
        # update error, weights and age of smallest
        error = self.G.nodes[f]['error']
        self.G.nodes[f]['error'] = error + dist[fi]
        self.G.nodes[f]['weight'] = weights[fi] + self.eta_f * diff[fi]
        if s in self.G[f]:
            self.G.edges[f, s]['age'] = 0
        else:
            self.G.add_edge(f, s, age = 0)
        # update weights and ages of the neighbors
        neighbors = [n for n in self.G[f]]
        nodes_to_update = []
        for n in neighbors:
            age = self.G.edges[f,n]['age']
            if age > self.max_age:
                self.G.remove_edge(f, n)
                if len(self.G[n]) == 0:
                    self.G.remove_node(n)
            else:
                self.G.edges[f,n]['age'] = age + 1
                nodes_to_update.append(n)
        for n in nodes_to_update:
            i = node_to_ind[n]
            self.G.nodes[n]['weight'] = weights[i] + self.eta_n * diff[i]

# my_gng = GNG(X_train)
# my_gng.training(50, 40)
# graphs = list(my_gng.G.subgraph(c) for c in nx.connected_components(my_gng.G))

# for g in graphs:
#     weights = []
#     for n in g.nodes:
#         weights.append(g.nodes[n]['weight'])
#     #print(np.asarray(weights).shape)
#     fxg, axs = plt.subplots(figsize=(12,12))
#     axs.plot(np.asarray(weights).T)
#     plt.show()

In [ ]:
# GNG modification
class GNG:
    def __init__(self, X):
        self.X = X
        self.samples, features = self.X.shape
        minn = self.X.min()
        maxx = self.X.max()
        self.G = nx.Graph()
        self.first_two(minn, maxx, features)
        self.new_node = 2
                        
        self.eta_f = 0.1
        self.eta_n = self.eta_f / 5
        self.max_age = 70
        self.use_factor = 1000
        self.biggest_use_reduction = 2
        self.global_use_reduction = 4
    
    def first_two(self, minn, maxx, features):
        for i in range(2):
            self.G.add_node(i, use = 0, weight = np.random.uniform(minn, maxx, (features)))
        self.G.add_edge(0, 1, age = 0)

    def training(self, epochs, lam):
        for epoch in range(epochs):
            sys.stdout.write('\r[%-50s] %d%%, epochs %d, nodes %d' % ('='*int(50 * (epoch+1) / epochs), 
                                                        100 * (epoch+1) / epochs, epoch+1, len(self.G.nodes)))
            sys.stdout.flush()
            order = np.random.permutation(self.samples)
            for i, ind in enumerate(order):
                self.update_net(ind, epoch)
                if i % lam == lam - 1:
                    self.create_neuron()
        sys.stdout.write('\n')
        
    def create_neuron(self):
        f = self.biggest_use()
        self.G.nodes[f]['use'] = self.G.nodes[f]['use'] / self.biggest_use_reduction
        neighbors = [n for n in self.G[f]]
        if len(neighbors) > 0:
            s = self.biggest_use(nodes = neighbors)
            self.G.nodes[s]['use'] = self.G.nodes[s]['use'] / self.biggest_use_reduction
            new_weight = (self.G.nodes[f]['weight'] + self.G.nodes[s]['weight']) / 2
            new_use = (self.G.nodes[f]['use'] + self.G.nodes[s]['use']) / 2
            self.G.add_node(self.new_node, use = new_use, weight = new_weight)
            self.G.add_edge(f, self.new_node, age = 0)
            self.G.add_edge(s, self.new_node, age = 0)
            self.G.remove_edge(f, s)
        else:
            new_weight = 0.9 * self.G.nodes[f]['weight']
            new_use = 0.9 * self.G.nodes[f]['use']
            self.G.add_node(self.new_node, use = new_use, weight = new_weight)
            self.G.add_edge(f, self.new_node, age = 0)
        self.new_node = self.new_node + 1
        
    def biggest_use(self, nodes = None):
        max_use = -1
        if nodes is None:
            for u in self.G.nodes:
                use = self.G.nodes[u]['use'] / self.global_use_reduction
                self.G.nodes[u]['use'] = use
                if use > max_use:
                    target = u
                    max_use = use
        else:
            for u in nodes:
                use = self.G.nodes[u]['use']
                if use > max_use:
                    target = u
                    max_use = use
        return target

    def update_net(self, ind, epoch):
        x = self.X[ind]
        min_0 = np.inf
        weights = []
        ind_to_node = []
        node_to_ind = {}
        for i, n in enumerate(self.G.nodes):
            ind_to_node.append(n)
            node_to_ind[n] = i
            weights.append(self.G.nodes[n]['weight'])
        # calculate distance and find 2 smallest
        diff = x - np.asarray(weights)
        dist = np.einsum('ij,ij->i', diff, diff)
        fi, si = np.argpartition(dist, 1)[:2]
        f = ind_to_node[fi]
        s = ind_to_node[si]
        # update use, weights and age of smallest
        max_use = self.G.nodes[f]['use']
        max_use = max_use + dist[si] - dist[fi]
        self.G.nodes[f]['use'] = max_use
        self.G.nodes[f]['weight'] = weights[fi] + self.eta_f * diff[fi]
        if s in self.G[f]:
            self.G.edges[f, s]['age'] = 0
        else:
            self.G.add_edge(f, s, age = 0)
        # update weights and ages of the neighbors
        neighbors = [n for n in self.G[f]]
        if epoch == 0:
            for n in neighbors:
                self.G.edges[f,n]['age'] = self.G.edges[f,n]['age'] + 1
                i = node_to_ind[n]
                self.G.nodes[n]['weight'] = weights[i] + self.eta_n * diff[i]
        else:
            nodes_to_update = []
            for n in neighbors:
                age = self.G.edges[f,n]['age']
                if age > self.max_age:
                    self.G.remove_edge(f, n)
                    if len(self.G[n]) == 0:
                        self.G.remove_node(n)
                else:
                    use = self.G.nodes[n]['use']
                    if use > max_use:
                        max_use = use
                    nodes_to_update.append(n)
            for n in nodes_to_update:
                use = self.G.nodes[n]['use']
                if max_use / self.use_factor > use:
                    self.G.remove_node(n)
                else:
                    i = node_to_ind[n]
                    self.G.edges[f,n]['age'] = age + 1
                    self.G.nodes[n]['weight'] = weights[i] + self.eta_n * diff[i]

#graphs = list(my_gng.G.subgraph(c) for c in nx.connected_components(my_gng.G))